In [ ]:
!pip install pyspark py4j

In [ ]:
import time
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigDataAnalytics").getOrCreate()

# Start time measurement
start_time = time.time()

In [ ]:
from pyspark.sql.functions import col, to_timestamp

# Initialize Spark session
spark = SparkSession.builder.appName("BigDataAnalytics").getOrCreate()

# Load datasets
weather_data = spark.read.csv("/content/Plant_1_Weather_Sensor_Data.csv", header=True, inferSchema=True)
generation_data = spark.read.csv("/content/Plant_1_Generation_Data.csv", header=True, inferSchema=True)

# Convert DATE_TIME column to standard format
weather_data = weather_data.withColumn("DATE_TIME", to_timestamp(col("DATE_TIME"), "yyyy-MM-dd HH:mm:ss"))
generation_data = generation_data.withColumn("DATE_TIME", to_timestamp(col("DATE_TIME"), "dd-MM-yyyy HH:mm"))

# Verify the change by showing the first few rows
weather_data.select("DATE_TIME").show(5)
generation_data.select("DATE_TIME").show(5)


+-------------------+
|          DATE_TIME|
+-------------------+
|2020-05-15 00:00:00|
|2020-05-15 00:15:00|
|2020-05-15 00:30:00|
|2020-05-15 00:45:00|
|2020-05-15 01:00:00|
+-------------------+
only showing top 5 rows

+-------------------+
|          DATE_TIME|
+-------------------+
|2020-05-15 00:00:00|
|2020-05-15 00:00:00|
|2020-05-15 00:00:00|
|2020-05-15 00:00:00|
|2020-05-15 00:00:00|
+-------------------+
only showing top 5 rows



Data Loading – Read and combine generation & weather datasets.

In [ ]:
# Rename SOURCE_KEY in both datasets before merging
generation_data = generation_data.withColumnRenamed("SOURCE_KEY", "GENERATION_SOURCE_KEY")
weather_data = weather_data.withColumnRenamed("SOURCE_KEY", "WEATHER_SOURCE_KEY")

# Merge datasets on DATE_TIME and PLANT_ID using join
merged_data = generation_data.join(weather_data, on=["DATE_TIME", "PLANT_ID"], how="inner")

# Show the first few rows of the merged data
merged_data.show(5)

# Save the merged dataset (saving in the same directory as initial data)
merged_data.write.mode("overwrite").option("header", "true").csv("/content/Merged_Plant_Data.csv")

+-------------------+--------+---------------------+--------+--------+-----------+-----------+------------------+-------------------+------------------+-----------+
|          DATE_TIME|PLANT_ID|GENERATION_SOURCE_KEY|DC_POWER|AC_POWER|DAILY_YIELD|TOTAL_YIELD|WEATHER_SOURCE_KEY|AMBIENT_TEMPERATURE|MODULE_TEMPERATURE|IRRADIATION|
+-------------------+--------+---------------------+--------+--------+-----------+-----------+------------------+-------------------+------------------+-----------+
|2020-05-15 00:00:00| 4135001|      1BY6WEcLGh8j5v7|     0.0|     0.0|        0.0|  6259559.0|   HmiyD2TTLFNqkNe| 25.184316133333333|        22.8575074|        0.0|
|2020-05-15 00:00:00| 4135001|      1IF53ai7Xc0U56Y|     0.0|     0.0|        0.0|  6183645.0|   HmiyD2TTLFNqkNe| 25.184316133333333|        22.8575074|        0.0|
|2020-05-15 00:00:00| 4135001|      3PZuoBAID5Wc2HD|     0.0|     0.0|        0.0|  6987759.0|   HmiyD2TTLFNqkNe| 25.184316133333333|        22.8575074|        0.0|
|2020-05-1

In [ ]:
from pyspark.sql import functions as F

df = merged_data  # Assign the merged_data DataFrame to df

# Calculate the count of missing (null) values per column
missing_values_count = df.select([F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns])

missing_values_count.show()


+---------+--------+---------------------+--------+--------+-----------+-----------+------------------+-------------------+------------------+-----------+
|DATE_TIME|PLANT_ID|GENERATION_SOURCE_KEY|DC_POWER|AC_POWER|DAILY_YIELD|TOTAL_YIELD|WEATHER_SOURCE_KEY|AMBIENT_TEMPERATURE|MODULE_TEMPERATURE|IRRADIATION|
+---------+--------+---------------------+--------+--------+-----------+-----------+------------------+-------------------+------------------+-----------+
|        0|       0|                    0|       0|       0|          0|          0|                 0|                  0|                 0|          0|
+---------+--------+---------------------+--------+--------+-----------+-----------+------------------+-------------------+------------------+-----------+



In [ ]:
# Calculate basic statistics
summary = df.select('DC_POWER', 'AC_POWER').describe()

# Show summary statistics (count, mean, stddev, min, max)
summary.show()

# Calculate specific percentiles (25%, 50%, 75%)
percentiles = [0.25, 0.5, 0.75]
percentile_values = df.approxQuantile(['DC_POWER', 'AC_POWER'], percentiles, 0.01)

# Displaying the percentiles
print("Percentiles (25%, 50%, 75%) for DC_POWER and AC_POWER:")
for i, col_name in enumerate(['DC_POWER', 'AC_POWER']):
    print(f"{col_name}: 25% = {percentile_values[i][0]}, 50% (median) = {percentile_values[i][1]}, 75% = {percentile_values[i][2]}")


+-------+------------------+------------------+
|summary|          DC_POWER|          AC_POWER|
+-------+------------------+------------------+
|  count|             68774|             68774|
|   mean|3147.1774501376367| 307.7783754808176|
| stddev|4036.4418255816813|394.39486476224266|
|    min|               0.0|               0.0|
|    max|         14471.125|           1410.95|
+-------+------------------+------------------+

Percentiles (25%, 50%, 75%) for DC_POWER and AC_POWER:
DC_POWER: 25% = 0.0, 50% (median) = 330.25, 75% = 6201.714286
AC_POWER: 25% = 0.0, 50% (median) = 31.9125, 75% = 607.5857143


In [ ]:
# Count zero values in the respective columns
zero_dc = df.filter(df["DC_POWER"] == 0).count()
zero_ac = df.filter(df["AC_POWER"] == 0).count()
zero_total_yield = df.filter(df["TOTAL_YIELD"] == 0).count()
zero_irradiation = df.filter(df["IRRADIATION"] == 0).count()

# Print the results
print(f"Zero DC Power: {zero_dc} instances")
print(f"Zero AC Power: {zero_ac} instances")
print(f"Zero Total Yield: {zero_total_yield} instances")
print(f"Zero Irradiation: {zero_irradiation} instances")


Zero DC Power: 31951 instances
Zero AC Power: 31951 instances
Zero Total Yield: 0 instances
Zero Irradiation: 30398 instances


Data Cleansing – Standardized DATE_TIME, removed redundant columns, and analyzed zero values.

In [ ]:
from pyspark.sql.functions import col, hour, when

# Extract hour from DATE_TIME
df = df.withColumn("Hour", hour(col("DATE_TIME")))

# List of columns to check
columns_to_check = ["DC_POWER", "AC_POWER", "TOTAL_YIELD", "IRRADIATION"]

# Create conditions to check for zero values in each column
zero_conditions = [when(col(c) == 0, 1).otherwise(0).alias(c) for c in columns_to_check]

# Add the zero value check columns to the DataFrame
df_with_zeros = df.select("Hour", *zero_conditions)

# Filter rows where any of the columns have zero values
zero_summary = df_with_zeros.filter(
    (col("DC_POWER") == 1) | (col("AC_POWER") == 1) |
     (col("TOTAL_YIELD") == 1) | (col("IRRADIATION") == 1)
)

# Group by Hour and count the number of zero values in each column
zero_count_by_hour = zero_summary.groupBy("Hour").agg(
    F.sum("DC_POWER").alias("DC_POWER"),
    F.sum("AC_POWER").alias("AC_POWER"),
    F.sum("TOTAL_YIELD").alias("TOTAL_YIELD"),
    F.sum("IRRADIATION").alias("IRRADIATION")
)

# Show the result
zero_count_by_hour.orderBy("Hour").show()


+----+--------+--------+-----------+-----------+
|Hour|DC_POWER|AC_POWER|TOTAL_YIELD|IRRADIATION|
+----+--------+--------+-----------+-----------+
|   0|    2724|    2724|          0|       2724|
|   1|    2726|    2726|          0|       2726|
|   2|    2810|    2810|          0|       2810|
|   3|    2812|    2812|          0|       2812|
|   4|    2815|    2815|          0|       2815|
|   5|    2707|    2707|          0|       2092|
|   6|     138|     138|          0|          0|
|   9|       2|       2|          0|          0|
|  10|       2|       2|          0|          0|
|  11|       6|       6|          0|          0|
|  12|      19|      19|          0|          0|
|  13|      27|      27|          0|          0|
|  14|       6|       6|          0|          0|
|  15|       1|       1|          0|          0|
|  18|     903|     903|          0|        248|
|  19|    2669|    2669|          0|       2587|
|  20|    2924|    2924|          0|       2924|
|  21|    2962|    2

We have added a new column by extracting Hour from DATE_TIME.

This will show:
- A table with hours (0-23) as rows.
- Counts of how many times each column had zero values at each hour.

Based on the results, most zero values appear at night (0:00 - 6:00, 18:00 - 23:00) and during some early morning or late afternoon hours when sunlight is weaker. This is expected for a solar power plant because:
- Nighttime (0:00 - 6:00, 19:00 - 23:00) → No sunlight → No power generation, no irradiation.
- Early morning (6:00 - 9:00) & late afternoon (15:00 - 18:00) → Low sunlight → Some zeros, but minimal.
- Daytime (10:00 - 14:00) → Few zeros → Power generation is good.

What this means for data cleaning:
1. Zeros at night (expected) → No need to remove them.
2. Zeros during daylight (rare but exists):
- Might be due to temporary weather conditions (clouds, rain).
- Might indicate sensor faults (unlikely, but possible).
- Since few zeros appear between 9:00 - 15:00, it’s not a major issue.


In [ ]:
# Drop GENERATION_SOURCE_KEY, WEATHER_SOURCE_KEY, and PLANT_ID columns
merged_data = merged_data.drop("GENERATION_SOURCE_KEY", "WEATHER_SOURCE_KEY", "PLANT_ID")

# Show the first few rows of the merged data to confirm changes
merged_data.show(5)

# Save the cleaned dataset to CSV (overwrite mode, header included)
merged_data.write.mode("overwrite").option("header", "true").csv("/content/Cleaned_Plant_Data.csv")

+-------------------+--------+--------+-----------+-----------+-------------------+------------------+-----------+
|          DATE_TIME|DC_POWER|AC_POWER|DAILY_YIELD|TOTAL_YIELD|AMBIENT_TEMPERATURE|MODULE_TEMPERATURE|IRRADIATION|
+-------------------+--------+--------+-----------+-----------+-------------------+------------------+-----------+
|2020-05-15 00:00:00|     0.0|     0.0|        0.0|  6259559.0| 25.184316133333333|        22.8575074|        0.0|
|2020-05-15 00:00:00|     0.0|     0.0|        0.0|  6183645.0| 25.184316133333333|        22.8575074|        0.0|
|2020-05-15 00:00:00|     0.0|     0.0|        0.0|  6987759.0| 25.184316133333333|        22.8575074|        0.0|
|2020-05-15 00:00:00|     0.0|     0.0|        0.0|  7602960.0| 25.184316133333333|        22.8575074|        0.0|
|2020-05-15 00:00:00|     0.0|     0.0|        0.0|  7158964.0| 25.184316133333333|        22.8575074|        0.0|
+-------------------+--------+--------+-----------+-----------+-----------------

What this code does:
1. Adds a new column: Power_Efficiency = AC_POWER / DC_POWER

2. Handles division by zero by filling NaN values with 0.

This is a column we additionaly added using columns AC_POWER & DC_POWER

In [ ]:
# Load cleaned dataset (infer schema to avoid string type issues)
df = spark.read.csv("Cleaned_Plant_Data.csv", header=True, inferSchema=True)

# Compute Power Efficiency safely (avoid division by zero)
df = df.withColumn(
    "Power_Efficiency",
    when(col("DC_POWER") != 0, col("AC_POWER") / col("DC_POWER")).otherwise(0)
)

# Save the updated dataset as a single CSV file
df.coalesce(1).write.csv("Cleaned_Plant_Data_Spark", header=True, mode="overwrite")

print("Power_Efficiency column added successfully!")


Power_Efficiency column added successfully!


Evaluate execution time

In [ ]:
import pandas as pd

# End time measurement
end_time = time.time()

# Print execution time
execution_time = end_time - start_time
print(f"Spark preprocessing execution time: {execution_time:.5f} seconds")

# Save execution time to CSV
execution_time_data = pd.DataFrame({'Framework': ['Spark'], 'Execution Time (seconds)': [execution_time]})
execution_time_data.to_csv('/content/execution_time_spark.csv', index=False)  # Save CSV file

Spark preprocessing execution time: 19.74609 seconds
